In [4]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [22]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/with-desciption/all_data_finetune_25mar.csv
/kaggle/input/all-data/all_data_finetune_23mar.csv


In [ ]:
%%capture
# Installs Unsloth, Xformers (Flash Attention) and all other packages!
!pip install unsloth
# Get latest Unsloth
!pip install --upgrade --no-deps "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

In [23]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/mistral-7b-v0.3-bnb-4bit",      # New Mistral v3 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/llama-3-8b-bnb-4bit",           # Llama-3 15 trillion tokens model 2x faster!
    "unsloth/llama-3-8b-Instruct-bnb-4bit",
    "unsloth/llama-3-70b-bnb-4bit",
    "unsloth/Phi-3-mini-4k-instruct",        # Phi-3 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit",             # Gemma 2.2x faster!
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

==((====))==  Unsloth 2025.3.18: Fast Llama patching. Transformers: 4.50.0.
   \\   /|    Tesla T4. Num GPUs = 2. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [24]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

In [70]:
from datasets import load_dataset
dataset = load_dataset(
    "csv",
    data_files = "/kaggle/input/all-data/all_data_finetune_23mar.csv",
    split = "train",
)
print(dataset.column_names)
print(dataset[0])

['sentiment', 'text']
{'sentiment': 1, 'text': 'K is-a P'}


In [26]:
print(dataset.column_names)

['sentiment', 'text', 'Document']


merged_prompt = \
        "[[The passenger embarked from {Embarked}.]]"\
        "[[\nThey are {Sex}.]]"\
        "[[\nThey have {Parch} parents and childen.]]"\
        "[[\nThey have {SibSp} siblings and spouses.]]"\
        "[[\nTheir passenger class is {Pclass}.]]"\
        "[[\nTheir age is {Age}.]]"\
        "[[\nThey paid ${Fare} for the trip.]]",

In [71]:
from unsloth import to_sharegpt
dataset = to_sharegpt(
    dataset,
    
    merged_prompt="[[Statement: {text}]]",
        
    conversation_extension = 5, # Randomnly combines conversations into 1! Good for long convos
    output_column_name = "sentiment",
)

In [27]:
from unsloth import to_sharegpt
dataset = to_sharegpt(
    dataset,
    
    merged_prompt="[[Document: {Document}.]]"
            "[[\nStatement: {text}.]]",
    conversation_extension = 5, 
    output_column_name = "sentiment",
)

Merging columns:   0%|          | 0/26 [00:00<?, ? examples/s]

Converting to ShareGPT:   0%|          | 0/26 [00:00<?, ? examples/s]

Flattening the indices:   0%|          | 0/26 [00:00<?, ? examples/s]

Flattening the indices:   0%|          | 0/26 [00:00<?, ? examples/s]

Flattening the indices:   0%|          | 0/26 [00:00<?, ? examples/s]

Flattening the indices:   0%|          | 0/26 [00:00<?, ? examples/s]

Flattening the indices:   0%|          | 0/26 [00:00<?, ? examples/s]

Extending conversations:   0%|          | 0/26 [00:00<?, ? examples/s]

In [72]:
from unsloth import standardize_sharegpt
dataset = standardize_sharegpt(dataset)

In [73]:
chat_template = """Analyze the below document. Determine whether the given statement is TRUE or FALSE.
Answer the corresponding sentiment label as "0" if FALSE or "1" if TRUE respectively.

<document1>
K is-a N; N is-a P; P is-a Q; Q is-a R; N is-a A; A is-a B; B is-a R; K is-a Z; F is-a Y; R isRelation FA; FA is-a ZA; R is-a XX
</document1>

Given statement:
{INPUT}

The validity of the statement is:
{OUTPUT}"""

from unsloth import apply_chat_template
dataset = apply_chat_template(
    dataset,
    tokenizer = tokenizer,
    chat_template = chat_template,
    # default_system_message = "You are a helpful assistant", << [OPTIONAL]
)

Unsloth: We automatically added an EOS token to stop endless generations.


Map:   0%|          | 0/16 [00:00<?, ? examples/s]

In [45]:
chat_template = """For each input analyze the document and state whether the given statement is TRUE or FALSE.
Each input contain a document and statement. In document, some relational information is given separated
by semicolon. Study each relations separated by semicolon in each document and then deduce whether the corresponding statement
is TRUE or FALSE.
Answer the corresponding sentiment label as "0" if FALSE or "1" if TRUE respectively.

Document & corresponding statement:
{INPUT}

The validity of the statement is:
{OUTPUT}"""

from unsloth import apply_chat_template
dataset = apply_chat_template(
    dataset,
    tokenizer = tokenizer,
    chat_template = chat_template,
   
)

Unsloth: We automatically added an EOS token to stop endless generations.


Map:   0%|          | 0/26 [00:00<?, ? examples/s]

In [74]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(report_to=None,
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Unsloth: We found double BOS tokens - we shall remove one automatically.


Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/16 [00:00<?, ? examples/s]

In [ ]:
"""%pip install -q wandb 
import wandb
wandb.login() """

In [75]:
#wandb.init()
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 16 | Num Epochs = 60 | Total steps = 60
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 4 x 1) = 16
 "-____-"     Trainable parameters = 41,943,040/8,000,000,000 (0.52% trained)


Step,Training Loss
1,1.148000
2,1.148000
3,0.868500
4,0.680700
5,0.512300
6,0.340200
7,0.220300
8,0.140800
9,0.105200
10,0.082700


In [57]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
messages = [                    # Change below!
    {"role": "user", "content": 'Document: H1 is-a H2; H2 is-a H3; H3 is-a H4; H5 isRelation H3; H3 is-a H6\n'\
                                'Statement: H4 is-a H2'},
]
input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True,
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids, streamer = text_streamer, max_new_tokens = 128, pad_token_id = tokenizer.eos_token_id)

0<|end_of_text|>


In [76]:
df_test=pd.read_csv('/kaggle/input/test-data/test_data_25mar.csv')
df_test.head()

,text,Document
0,H1 is-a H5,H1 is-a H2; H2 is-a H3; H3 is-a H4; H5 isRelat...
1,H5 is-a H3,H1 is-a H2; H2 is-a H3; H3 is-a H4; H5 isRelat...
2,H6 isRelation H3,H1 is-a H2; H2 is-a H3; H3 is-a H4; H5 isRelat...
3,H2 is-a H4,H1 is-a H2; H2 is-a H3; H3 is-a H4; H5 isRelat...
4,H6 is-a H1,H1 is-a H2; H2 is-a H3; H3 is-a H4; H5 isRelat...


In [77]:
from transformers import TextStreamer

answers_list=[]

for i in range(len(df_test)):
    aj1_doc=df_test.iloc[i,1]
    aj1_text=df_test.iloc[i,0]
    #print(df_test.iloc[i,0])
    messages = [                    # Change below!
    {"role": "user", "content": f"""
    Analyze document and state whether the corresponding statement is TRUE or FALSE.
    Document: {aj1_doc}
                                
  'Statement: {aj1_text}"""},
        ]
    input_ids = tokenizer.apply_chat_template(
       messages,
       add_generation_prompt = True,
       return_tensors = "pt",
        ).to("cuda")


    text_streamer = TextStreamer(tokenizer, skip_prompt = True)
    a1 = model.generate(input_ids, streamer = text_streamer, max_new_tokens = 128, pad_token_id = tokenizer.eos_token_id)
    answers_list.append(a1)
    

1<|end_of_text|>
1<|end_of_text|>
0<|end_of_text|>
0<|end_of_text|>
0<|end_of_text|>
0<|end_of_text|>
1<|end_of_text|>
1<|end_of_text|>
0<|end_of_text|>
1<|end_of_text|>
0<|end_of_text|>


In [ ]:
For the doc-text
0<|end_of_text|>
0<|end_of_text|>
0<|end_of_text|>
0<|end_of_text|>
0<|end_of_text|>
0<|end_of_text|>
1<|end_of_text|>
0<|end_of_text|>
0<|end_of_text|>
0<|end_of_text|>
0<|end_of_text|>

In [ ]:
only text
1<|end_of_text|>
1<|end_of_text|>
0<|end_of_text|>
0<|end_of_text|>
0<|end_of_text|>
0<|end_of_text|>
1<|end_of_text|>
1<|end_of_text|>
0<|end_of_text|>
1<|end_of_text|>
0<|end_of_text|>

In [64]:
print(answers_list)

NameError: name 'answers_list' is not defined

In [63]:
'0<|end_of_text|>'.split('<')[0]

'0'